# 🦎 Youtube Video Downloader 🦎

## 🐍 Encubadora 🐍

In [16]:
# import pytube

# url = 'https://www.youtube.com/watch?v=eycU0vO9Gzc'

# yt = pytube.YouTube(url)

# video = yt.streams.first()

# video.download(r"C:\Users\matheus_meissner\Desktop\github\yt_video_downloader")

## 🐢 Depuração 🐢

### Bibliotecas

In [17]:
import pytube

### Define o URL do vídeo

In [18]:
url = 'https://www.youtube.com/watch?v=VTu1kJX52mU'

### Cria um objeto YouTube com o URL do vídeo

In [19]:
yt = pytube.YouTube(url)

### Seleciona o primeiro stream de vídeo disponível

In [20]:
video = yt.streams.first()

### Baixa o vídeo

In [21]:
video.download(r"C:\Users\matheus_meissner\Desktop\github\yt_video_downloader")

'C:\\Users\\matheus_meissner\\Desktop\\github\\yt_video_downloader\\🔴 LIVE DO PROJETO 3 - A MAGIA DO JAVASCRIPT - FINALIZANDO O PROJETO E ADICIONANDO AO GITHUB.mp4'